In [56]:
import numpy as numpy
import pandas as pd
from sklearn.model_selection import KFold,RandomizedSearchCV

In [73]:
train=pd.read_csv("train (1).csv")
test=pd.read_csv("test (1).csv")
print(train.shape)
print(test.shape)

(149087, 21)
(63895, 20)


In [74]:
target=train['Class']
train.drop(labels=['Class'],axis=1,inplace=True)

In [75]:
full_data=pd.concat([train,test],sort=False,ignore_index=True)

In [76]:
full_data.drop(labels=['ID','Year Type','Employee Identifier','Organization Group','Department Code','Job Family','Union Code','Job Code'],axis=1,inplace=True)

In [20]:
dummies=pd.get_dummies(full_data['Department'],prefix='Dept')
full_data=pd.concat([full_data,dummies],axis=1)

dummies=pd.get_dummies(full_data['Union'],prefix='Union')
full_data=pd.concat([full_data,dummies],axis=1)

dummies=pd.get_dummies(full_data['Job Family Code'],prefix='Job_family_code')
full_data=pd.concat([full_data,dummies],axis=1)

dummies=pd.get_dummies(full_data['Job'],prefix='Job')
full_data=pd.concat([full_data,dummies],axis=1)    

In [21]:
full_data.drop(labels=['Department','Union','Job Family Code','Job'],axis=1,inplace=True)

In [22]:
from sklearn.ensemble import RandomForestClassifier

In [23]:
train_X=full_data.iloc[:train.shape[0],:]
train_y=target

In [24]:
test_X=full_data.iloc[train.shape[0]:,:]

In [71]:
model=RandomForestClassifier(n_estimators=1000)

In [72]:
model.fit(train_X,train_y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

import xgboost as xgb
from xgboost.sklearn import XGBClassifier
model = xgb.XGBClassifier()

xgdmat=xgb.DMatrix(train_X,train_y)
xgb_model=xgb.XGBRegressor()
#our_params={'eta':0.1,'subsample':0.7,'colsample_bytree':0.7,'objective':'logloss','max_depth':4,'min_child_weight':5}
final_gb=xgb.train(xgdmat)
tesdmat=xgb.DMatrix(test_X)
y_pred=final_gb.predict(tesdmat)

In [63]:
pred=model.predict(test_X)
pred

array([1, 3, 2, ..., 2, 2, 3], dtype=int64)

In [64]:
df = pd.DataFrame(data=pred)

In [65]:
df.to_excel("final_sub.xlsx")

In [66]:
model.score(train_X,train_y)

0.9164045154842475

In [54]:
from sklearn.model_selection import GridSearchCV

In [ ]:
#model = xgb.XGBClassifier()
import time
kf = KFold(n_splits=5, shuffle=True, random_state=30)
for dev_index, val_index in kf.split(train_X, train_y): 
    dev_X, val_X = train_X.loc[dev_index,:], train_X.loc[val_index,:]
    dev_y, val_y = train_y[dev_index], train_y[val_index]
   
    param_grid = {
     'max_depth': [2,3,4,5,6,7,8],
    'n_estimators':[500,800,1000,2000,3000],
    'min_samples_leaf':[1,10,50,100],
    'min_samples_split':[1,10,100,1000]    
      }
 
#fit_params = {'eval_metric': 'neg_mean_squared_error','early_stopping_rounds': 10,'eval_set': [(val_X,val_y)]}
 
    rs_clf = RandomizedSearchCV(model, param_grid, n_iter=20,n_jobs=1 ,refit=False, random_state=42)
 
    print("Randomized search..")
    search_time_start = time.time()
    rs_clf.fit(dev_X,dev_y)
    print("Randomized search time:", time.time() - search_time_start)
    best_score = rs_clf.best_score_
    best_params = rs_clf.best_params_
    print("Best score: {}".format(best_score))
    print("Best params: ")
    for param_name in sorted(best_params.keys()):
        print('%s: %r' % (param_name, best_params[param_name]))

Randomized search..
